In [2]:
%run ../../../main.py
%matplotlib inline

In [4]:
import pandas as pd

from sklearn.metrics import accuracy_score

from pyarc.algorithms import M1Algorithm, M2Algorithm, top_rules, createCARs 
from pyarc.data_structures import TransactionDB


import pyarc.qcba as qcba
from pyarc import CBA
from pyarc.qcba.data_structures import *
import matplotlib.pyplot as plt

from pyarc.qcba import QCBA

import pandas as pd
from pyarc.qcba.data_structures import (
    IntervalReader,
    Interval,
    QuantitativeDataFrame,
    QuantitativeCAR
)

interval_reader = IntervalReader()

interval_reader.closed_bracket = "", "NULL"
interval_reader.open_bracket = "NULL", ""
interval_reader.infinity_symbol = "inf", "inf"
interval_reader.members_separator = "_to_"

interval_reader.compile_reader()

i = interval_reader.read("82.9815_to_inf")

QuantitativeCAR.interval_reader = interval_reader

In [20]:
directory = "c:/code/python/machine_learning/assoc_rules"

def func(datasetname, unique_transactions=True):
    pd_ds = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/{}.csv".format(datasetname))
    
    
    pd_ds_undiscr = pd.read_csv("c:/code/python/machine_learning/assoc_rules/folds_undiscr/train/{}.csv".format(datasetname))
    pd_ds_undiscr_test = pd.read_csv("c:/code/python/machine_learning/assoc_rules/folds_undiscr/test/{}.csv".format(datasetname))
    
    txns = TransactionDB.from_DataFrame(pd_ds, unique_transactions=unique_transactions)
    txns_test = TransactionDB.from_DataFrame(pd.read_csv("c:/code/python/machine_learning/assoc_rules/test/{}.csv".format(datasetname)))

    
    rules = top_rules(txns.string_representation, appearance=txns.appeardict)

    rules.sort(reverse=True)


    cars = createCARs(rules)

    cars.sort(reverse=True)

    if len(cars) > 1000:
        cars = cars[:1000]
        

    print("len(rules)", len(cars))

    m1 = M1Algorithm(cars, txns)
    
    m2 = M2Algorithm(cars, txns)
    
    m1clf = m1.build()
    m2clf = m2.build()
    
    rm_cba = CBA()
    rm_cba.clf = m1clf
    
    quant_dataframe = QuantitativeDataFrame(pd_ds_undiscr)
    
    rm_qcba = QCBA(rm_cba, quant_dataframe)
    qcba_clf = rm_qcba.fit()
    
    actual = list(map(lambda i: i.value, txns_test.class_labels))

    pred = m1clf.predict_all(txns_test)
    predM2 = m2clf.predict_all(txns_test)
    
    accM2 = accuracy_score(predM2, actual)
    acc = accuracy_score(pred, actual)
    acc_qcba = qcba_clf.rule_model_accuracy(QuantitativeDataFrame(pd_ds_undiscr_test), pd_ds_undiscr_test.iloc[:,-1])

    return acc, accM2, acc_qcba



def mean_func(dataset_name, start=0, end=10, unique_transactions=True):
    files = [ dataset_name + repr(i) for i in range(start, end) ]

    accs = []
    accsM2 = []
    accs_qcba = []
    
    for file in files:
        acc, accM2, acc_qcba = func(file, unique_transactions=unique_transactions)
        print("done", file, acc)
        print("done m2", file, accM2)
        print("done qcba", file, acc_qcba)
        accs.append(acc)
        accsM2.append(accM2)
        accs_qcba.append(acc_qcba)
        
    mn = sum(accs) / len(accs)
    mnM2 = sum(accsM2) / len(accsM2)
    mn_qcba = sum(accs_qcba) / len(accs_qcba)
    
    return mn, mnM2, mn_qcba


                
datasets = [
    "iris",
    "breast-w",
    "anneal",
    "hypothyroid",
    "ionosphere",
    "lymph",
    "vehicle",
    "autos",
    "diabetes",
    "glass",
    "heart-h",
    "tic-tac-toe",
    "australian",
    "sick",
    "segment",
    "spambase",
    "sonar",
    "vowel",
    "hepatitis",
    "credit-a",
    "mushroom",
    "house-votes-84",
    "soybean",
    "primary-tumor",
    "credit-g",
    "audiology",
    "breast-cancer",
    "balance-scale",
    "heart-c",
    "kr-vs-kp",
    "pima",
    "heart-statlog"] 

datasets = [
    "heart-statlog",
    "iris"
]

means = []
meansM2 = []
means_qcba = []
for dataset in ["autos"]:
    acc, accM2, acc_qcba = mean_func(dataset)
    print("*****")
    print("M1", dataset, acc)
    print("M2", dataset, accM2)
    print("******")
    
    means.append((dataset, acc))
    meansM2.append((dataset, accM2))
    means_qcba.append((dataset, acc_qcba))
    
print("M1")
print(means)
print("\nM2")
print(meansM2)
print("\nQCBA")
print(means_qcba)

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=26
Rule count: 8294, Iteration: 1
Target rule count satisfied: 1000
len(rules) 1000
applying selected transformations
refitting
literal pruning


C:\code\python\CBA\pyarc\qcba\data_structures\quant_dataset.py:247: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = class_column == value


AttributeError: 'bool' object has no attribute 'reshape'